# Lazy Learning with k-Nearest Neighbors {#knn}

<iframe width="560" height="315" src="https://www.youtube.com/embed/MDniRwXizWo" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen>

</iframe>

K-nearest neighbors is an algorithm that is commonly used in data mining. It works by identifying the k-nearest data points to a given point, and using their values to predict the value of the point in question.

```{r echo=FALSE, message=TRUE, warning=TRUE}
index <- c("2")
topic <- c("K-Nearest Neighbors")
tasks <- c("Build a k-nearest neighbors model and explain how it may be used to predict the values of data points.")
activities <- c("Follow along: students will participate in a guided demo of a data mining process building a model using K-Nearest Neighbors and evaluating its accuracy using a Confusion Matrix.")
student <- c("'We learned about the KNN algorithm, its advantages and limitations, as well as how to interpret a confusion matrix to evaluate the accuracy of a model.'")
teacher <- c("'Our goal here is to understand how data mining algorithms work and how they can be applied to real-world problems. As a teacher, my role is to clarify any doubts and ensure that everyone is actively participating. As students, you will be challenged to apply your knowledge to a problem and think critically.'")
headers <- c("","Topic", "Tasks", "Activities", "Student", "Teacher")
outlineDF <- data.frame(index, topic, tasks, activities, student, teacher)
colnames(outlineDF) <- headers

knitr::kable(
  outlineDF, longtable = TRUE, booktabs = TRUE,
  caption = 'Lesson outline'
)
```

In [ ]:
import pandas as pd
from pandas.plotting import scatter_matrix
from pandas.api.types import CategoricalDtype
from IPython.display import display, Markdown
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

## Business Case: Diagnosing Breast Cancer

Breast cancer is the top cancer in women both in the developed and the developing world. In the Netherlands it is the most pervasive form of cancer [@noauthor_who_nodate]. In order to improve breast cancer outcome and survival early detection remains the most important instrument for breast cancer control. If machine learning could automate the identification of cancer, it would improve efficiency of the detection process and might also increase its effectiveness by providing greater detection accuracy.

## Data Understanding

The data we will be using comes from the University of Wisconsin and is available online as an open source dataset [@noauthor_uci_cancer_nodate]. It includes measurements from digitized images from from fine-needle aspirates of breast mass. The values represent cell nuclei features.

For convenience the data in csv format is stored on Github. We can access it directly using a function for reading csv from the `pandas` library

In [ ]:
url = "https://raw.githubusercontent.com/businessdatasolutions/courses/main/data%20mining/gitbook/datasets/breastcancer.csv"
rawDF = pd.read_csv(url)

Using the `info()` function we can have some basic information about the dataset.

In [ ]:
rawDF.info()

We can also check visually for patterns.

In [ ]:
selDF = rawDF.filter(regex="mean").iloc[:, :6]
fig = scatter_matrix(selDF, alpha=0.2, figsize=(6, 6), diagonal="hist")
for ax in fig.flatten():
    ax.xaxis.label.set_rotation(90)
    ax.yaxis.label.set_rotation(0)
    ax.yaxis.label.set_ha("right")
plt.tight_layout()
plt.gcf().subplots_adjust(wspace=0, hspace=0)
plt.show()

**Question:**

-   *What patterns do you observe and to what extend are they surprising?*

## Preparation

The first variable, `id`, contains unique patient IDs. The IDs do not possess any relevant information for making predictions, so we will delete it from the dataset.

In [ ]:
cleanDF = rawDF.drop(["id"], axis=1)
cleanDF.head()

The variable named `diagnosis` contains the outcomes we would like to predict - 'B' for 'Benign' and 'M' for 'Malignant'. The variable we would like to predict is called the 'label'. We can look at the counts for both outcomes, using the `value_counts()` function. When we set the normalize setting to `True` we get the the proportions.

In [ ]:
cntDiag = cleanDF["diagnosis"].value_counts()
propDiag = cleanDF["diagnosis"].value_counts(normalize=True)
cntDiag
propDiag

Looking again at the results from the `info()` function you'll notice that the variable `diagnosis` is coded as text (`object`). Many models require that the label is of type `category`. The `pandas` library has a function that can transform a `object` type to `category`.

In [ ]:
catType = CategoricalDtype(categories=["B", "M"], ordered=False)
cleanDF["diagnosis"] = cleanDF["diagnosis"].astype(catType)
cleanDF["diagnosis"]

The features consist of three different measurements of ten characteristics. We will take three characteristics and have a closer look.

In [ ]:
cleanDF[["radius_mean", "area_mean", "smoothness_mean"]].describe()

You'll notice that the three variables have very different ranges and as a consequence `area_mean` will have a larger impact on the distance calculation than the `smootness_mean`. This could potentially cause problems for modeling. To solve this we'll apply normalization to rescale all features to a standard range of values.

We will write our own normalization function,

In [ ]:
def normalize(x):
    return (x - min(x)) / (
        max(x) - min(x)
    )  # distance of item value - minimum vector value divided by the range of all vector values


testSet1 = np.arange(1, 6)
testSet2 = np.arange(1, 6) * 10


print(f"testSet1: {testSet1}\n")
print(f"testSet2: {testSet2}\n")
print(f"Normalized testSet1: {normalize(testSet1)}\n")
print(f"Normalized testSet2: {normalize(testSet2)}\n")

and apply it to all the numerical variables in the dataframe.

In [ ]:
excluded = ["diagnosis"]  # list of columns to exclude
# X = cleanDF.loc[:, ~cleanDF.columns.isin(excluded)]
X = cleanDF.drop(excluded, axis=1)
X = X.apply(normalize, axis=0)
X[["radius_mean", "area_mean", "smoothness_mean"]].describe()

When we take the variables we've selected earlier and look at the summary parameters again, we'll see that the normalization was successful.

We can now split our data into training and test sets.

In [ ]:
y = cleanDF["diagnosis"]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

Here, X_train and y_train are the features and labels of the training data, respectively, and X_test and y_test are the features and labels of the test data.

Now we can train and evaluate our kNN model.

## Modeling and Evaluation

KNN is a instance-based learning algorithm. It stores all of the training data and makes predictions based on the similarity between the input instance and the stored instances. The prediction is based on the majority class among the K nearest neighbors of the input instance.

The distance between instances is typically measured using the Euclidean distance. However, other distance measures such as the Manhattan distance or the Minkowski distance can also be used.

The pseudocode for the KNN algorithm is as follows:

::: {.p-2 style="background-color:#f0f3f4;"}
```{=html}
<pre><code class=''>
<span class="hljs-keyword">for</span> <span class="hljs-keyword">each</span> instance <span class="hljs-keyword">in</span> the test <span class="hljs-keyword">set</span>:
    <span class="hljs-keyword">for</span> <span class="hljs-keyword">each</span> instance <span class="hljs-keyword">in</span> the training <span class="hljs-keyword">set</span>:
        calculate the distance between the two instances
    sort the distances <span class="hljs-keyword">in</span> ascending <span class="hljs-keyword">order</span>
    find the K nearest neighbors
    predict the <span class="hljs-keyword">class</span> based <span class="hljs-keyword">on</span> the majority <span class="hljs-keyword">class</span> among the K nearest neighbors
</code></pre>
```
:::

To train the knn model we only need one single function from the `sklearn` library. The `fit()` function trains the model on the training data. The trained model is applied to the set with test features and the `predict()` function gives back a set of predicted values for y.

NOTE: Somehow `predict()` generates error with number columns larger than 15. This is a new issue. It used to work.

In [ ]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train.iloc[:,:15], y_train)
# # make predictions on the test set
y_pred = knn.predict((X_test.iloc[:,:15]))

Now that we have a set of predicted labels we can compare these with the actual labels. A diffusion table shows how well the model performed.

```{r difftable-fig, echo=FALSE, fig.align='center', fig.asp=.75, fig.cap='Standard diffusion table. Taken from: https://emj.bmj.com/content/emermed/36/7/431/F1.large.jpg', message=TRUE, warning=TRUE, out.width='80%'}
knitr::include_graphics(rep('images/diffusion.png'))
```

Here is our own table:

In [ ]:
cm = confusion_matrix(y_test, y_pred, labels=knn.classes_)
cm

In [ ]:
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=knn.classes_)
disp.plot()
plt.show()

**Questions:**

1.  *How would you assess the overall performance of the model?*
2.  *What would you consider as more costly: high false negatives or high false positives levels? Why?*
3.  *Try to improve the model by changing some parameters of the `KNeighborsClassifier()` function*